In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mlaut.data import Data
from mlaut.estimators.estimators import instantiate_default_estimators
from mlaut.experiments import Orchestrator
from mlaut.analyze_results import AnalyseResults
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets


/media/viktor/Data/PhD/mlaut/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Step 1: Download the datasets

In [2]:
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False)

Error: Dataset Delgado_data/molec-biol-protein-second has a different number of arff files


### Step 2: Define Input and Output HDF5 objects

In [3]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='a')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

### Step 3: Save the datasets in HDF5 database

In [4]:
# files io object needs to be used in test orch
# returned object needs to be integrated in code

data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, input_io=input_io)

### Step 4: Split datasets

In [5]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path)

2018-03-18 21:42:41,313 [MainThread  ] [WARNI]  Skipping abalone as test/train split already exists in output h5 file.
2018-03-18 21:42:41,327 [MainThread  ] [WARNI]  Skipping acute_inflammation as test/train split already exists in output h5 file.
2018-03-18 21:42:41,343 [MainThread  ] [WARNI]  Skipping acute_nephritis as test/train split already exists in output h5 file.
2018-03-18 21:42:41,361 [MainThread  ] [WARNI]  Skipping adult as test/train split already exists in output h5 file.
2018-03-18 21:42:41,381 [MainThread  ] [WARNI]  Skipping annealing as test/train split already exists in output h5 file.
2018-03-18 21:42:41,394 [MainThread  ] [WARNI]  Skipping arrhythmia as test/train split already exists in output h5 file.
2018-03-18 21:42:41,415 [MainThread  ] [WARNI]  Skipping audiology_std as test/train split already exists in output h5 file.
2018-03-18 21:42:41,431 [MainThread  ] [WARNI]  Skipping balance_scale as test/train split already exists in output h5 file.
2018-03-18 21:

2018-03-18 21:42:42,557 [MainThread  ] [WARNI]  Skipping nursery as test/train split already exists in output h5 file.
2018-03-18 21:42:42,572 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_nucleus_4d as test/train split already exists in output h5 file.
2018-03-18 21:42:42,592 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_states_2f as test/train split already exists in output h5 file.
2018-03-18 21:42:42,610 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_nucleus_2f as test/train split already exists in output h5 file.
2018-03-18 21:42:42,629 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_states_5b as test/train split already exists in output h5 file.
2018-03-18 21:42:42,647 [MainThread  ] [WARNI]  Skipping optical as test/train split already exists in output h5 file.
2018-03-18 21:42:42,660 [MainThread  ] [WARNI]  Skipping ozone as test/train split already exists in output h5 file.
2018-03-18 21:42:42,678 [MainThread  ] [WARNI]  Skipping page_blocks as test/trai

### Step 5: Instantiate estimator objects and the experiments orchestrator class.

In [6]:
instantiated_models = instantiate_default_estimators(estimators=['NeuralNetworkDeepClassifier'])

orchest = Orchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io, dts_names=dts_names_list[0:2],
                 original_datasets_group_h5_path='delgado_datasets/')

### Step 6: Run the experiments

In [7]:
orchest.run(modelling_strategies=instantiated_models)

*** Training models on dataset: abalone. Total datasets processed: 1/2 ***


ValueError: cannot reshape array of size 8394 into shape (2798,1)

### Step 7: Make predictions on the test sets

In [ ]:
orchest.predict_all(trained_models_dir='data/trained_models', estimators=instantiated_models)

### Step 8: Analyze the results

In [ ]:
analyze = AnalyseResults(hdf5_output_io=out_io, 
                         hdf5_input_io=input_io,
                         input_h5_original_datasets_group='delgado_datasets/', 
                         output_h5_predictions_group='experiments/predictions/')
error_all_datasets = analyze.calculate_error_all_datasets(metric='mean_squared_error')
# observations = analyze.calculate_loss_all_datasets(input_h5_original_datasets_group='delgado_datasets/', 
#                                     output_h5_predictions_group='experiments/predictions/', 
#                                     metric='mean_squared_error')

#### t-test

In [ ]:
t_test, t_test_df = analyze.t_test(error_all_datasets)
t_test_df

#### sign test

In [ ]:
sign_test, sign_test_df = analyze.sign_test(error_all_datasets)
sign_test_df

#### t-test with bonferroni correction

In [ ]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(error_all_datasets)
t_test_bonferroni_df

#### Wilcoxon test

In [ ]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(error_all_datasets)
wilcoxon_test_df

#### Friedman test

In [ ]:
friedman_test, friedman_test_df = analyze.friedman_test(error_all_datasets)
friedman_test_df

#### Nemenyi test

In [ ]:
nemeniy_test = analyze.nemenyi(error_all_datasets)
nemeniy_test

In [ ]:
nemeniy_test = analyze.nemenyi(error_all_datasets)
nemeniy_test